### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [97]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
sdc_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
sdc_df.dtypes
sdc_df.count()
sdc_df.columns
sdc_df.head()
school_data.set_index('school_name')

School ID      type  size   budget
school_name                                              
Huang High School              0  District  2917  1910635
Figueroa High School           1  District  2949  1884411
Shelton High School            2   Charter  1761  1056600
Hernandez High School          3  District  4635  3022020
Griffin High School            4   Charter  1468   917500
Wilson High School             5   Charter  2283  1319574
Cabrera High School            6   Charter  1858  1081356
Bailey High School             7  District  4976  3124928
Holden High School             8   Charter   427   248087
Pena High School               9   Charter   962   585858
Wright High School            10   Charter  1800  1049400
Rodriguez High School         11  District  3999  2547363
Johnson High School           12  District  4761  3094650
Ford High School              13  District  2739  1763916
Thomas High School            14   Charter  1635  1043130

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# Calculate summary metrics
school_count = len(sdc_df['school_name'].unique())
student_count = len(sdc_df['student_name'])
total_budget = sdc_df['budget'].unique().sum()
ave_math_score = sdc_df['math_score'].mean()
ave_reading_score = sdc_df['reading_score'].mean()
math_pass_score_count = len(sdc_df[sdc_df['math_score']>=70])
math_pass_percent = (math_pass_score_count/student_count) * 100
reading_pass_score_count = len(sdc_df[sdc_df['reading_score']>=70])
reading_pass_percent = (reading_pass_score_count/student_count) * 100
overall_pass_count = len(sdc_df[(sdc_df['math_score']>=70) & (sdc_df['reading_score']>=70)])
overall_pass_percent = (overall_pass_count/student_count)*100

#I know that I did not need to print these but I did it to verify my results befor putting them the the df
print(school_count)
print(student_count)
print(total_budget)
print(ave_math_score)
print(ave_reading_score)
print(math_pass_percent)
print(reading_pass_percent)
print(overall_pass_percent)


dist_sum_df = pd.DataFrame({
    'Number of Schools':[school_count], 
    'Number of Students':[student_count], 
    'Total Budget':[total_budget], 
    'Average Math Score':[ave_math_score], 
    'Average English Score':[ave_reading_score],
    '% Passing Math':[math_pass_percent],
    '% Passing English':[reading_pass_percent],
    '% Overall Passing': [overall_pass_percent]   
})

#dist_sum_df['Total Budget'] = dist_sum_df['Total Budget'].map('${:.2f}'.format)
dist_sum_df['Total Budget'] = dist_sum_df['Total Budget'].map("{:,}".format)
dist_sum_df

15
39170
24649428
78.98537145774827
81.87784018381414
74.9808526933878
85.80546336482001
65.17232575950983


Number of Schools  Number of Students Total Budget  Average Math Score  \
0                 15               39170   24,649,428           78.985371   

   Average English Score  % Passing Math  % Passing English  % Overall Passing  
0               81.87784       74.980853          85.805463          65.172326

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [120]:
schoolgroup = sdc_df.groupby(['type','school_name'])
schooltype = schoolgroup['math_score'].sum()
schooltype

type      school_name          
Charter   Cabrera High School      154329
          Griffin High School      122360
          Holden High School        35784
          Pena High School          80654
          Shelton High School      146796
          Thomas High School       136389
          Wilson High School       190115
          Wright High School       150628
District  Bailey High School       383393
          Figueroa High School     226223
          Ford High School         211184
          Hernandez High School    358238
          Huang High School        223528
          Johnson High School      366942
          Rodriguez High School    307294
Name: math_score, dtype: int64

In [107]:
total_students = sdc_df['school_name'].value_counts()
total_students_df = pd.DataFrame(total_students)

grouped_schools = sdc_df.groupby(['school_name'])
school_type = grouped_schools['type'].unique()
school_type_df = pd.DataFrame(school_type)
#school_type_df['type'] = school_type_df.loc[:, 'type'].astype(object)

school_budget = grouped_schools['budget'].mean()
school_budget_df = pd.DataFrame(school_budget)
school_budget_df['budget'] = school_budget_df.loc[:, 'budget'].astype(float)

budget_per_student = school_budget/total_students
budget_per_student_df = pd.DataFrame(budget_per_student)
budget_per_student_df.columns = ['Budget Per Student']

math_average = grouped_schools['math_score'].mean()
math_average_df = pd.DataFrame(math_average)

reading_average = grouped_schools['reading_score'].mean()
reading_average_df = pd.DataFrame(reading_average)


school_summary_df = pd.concat([school_type, total_students_df, school_budget_df, budget_per_student_df, math_average_df,
reading_average_df], axis=1, join='inner')
school_summary_df = school_summary_df.rename(columns={"type": "School Type", 'school_name':'Total Students', "budget":"Total School Budget", "math_score": "Average Math Score", "reading_score":"Average Reading Score"})
school_summary_df['School_Type'] = school_summary_df


29370

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [121]:
top_five_df = school_summary_df.sort_values('Total School Budget', ascending=False)
top_five_df.head()


School Type  Total Students  Total School Budget  \
Bailey High School     [District]            4976            3124928.0   
Johnson High School    [District]            4761            3094650.0   
Hernandez High School  [District]            4635            3022020.0   
Rodriguez High School  [District]            3999            2547363.0   
Huang High School      [District]            2917            1910635.0   

                       Budget Per Student  Average Math Score  \
Bailey High School                  628.0           77.048432   
Johnson High School                 650.0           77.072464   
Hernandez High School               652.0           77.289752   
Rodriguez High School               637.0           76.842711   
Huang High School                   655.0           76.629414   

                       Average Reading Score  
Bailey High School                 81.033963  
Johnson High School                80.966394  
Hernandez High School              80.934412  
Rodriguez High School              80.744686  
Huang High School                  81.182722

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [11]:
Bottom_five_df = school_summary_df.sort_values('Total Students')
Bottom_five_df.head()

School Type  Total Students  Total School Budget  \
Holden High School    [Charter]             427             248087.0   
Pena High School      [Charter]             962             585858.0   
Griffin High School   [Charter]            1468             917500.0   
Thomas High School    [Charter]            1635            1043130.0   
Shelton High School   [Charter]            1761            1056600.0   

                     Budget Per Student  Average Math Score  \
Holden High School                581.0           83.803279   
Pena High School                  609.0           83.839917   
Griffin High School               625.0           83.351499   
Thomas High School                638.0           83.418349   
Shelton High School               600.0           83.359455   

                     Average Reading Score  
Holden High School               83.814988  
Pena High School                 84.044699  
Griffin High School              83.816757  
Thomas High School               83.848930  
Shelton High School              83.725724

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [88]:
#school_grade_group = sdc_df.groupby(['school_name', 'grade'])
#math_score_by_grade = school_grade_group['math_score'].mean()
#math_score_by_grade_df = pd.DataFrame(math_score_by_grade)
#math_score_by_grade_df
#math_score_by_grade_df.unstack('grade')
#math_score_by_grade_df.columns = ['school_name',]



math_score_by_grade = sdc_df[['grade','school_name','math_score']].groupby(['grade','school_name']).mean()
math_score_by_grade

math_score_by_grade_df = pd.DataFrame(math_score_by_grade)
math_score_by_grade_df
math_score_by_grade.unstack('grade')

#math_by_grade_df = math_score_by_grade_df.set_index(['school_name'] + ['9th', '10th', '11th', '12th']).unstack('grade')

#new_df = df.set_index(['name'] + metas).unstack('name'))

#math_score_by_grade_df=math_score_by_grade_df['9th','10th','11th','12th']

math_score                                 
grade                       10th       11th       12th        9th
school_name                                                      
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [82]:
reading_score_by_grade = sdc_df[['grade','school_name','reading_score']].groupby(['school_name', 'grade']).mean()
reading_score_by_grade_df = pd.DataFrame(reading_score_by_grade)
reading_score_by_grade_df

reading_score_by_grade_df.unstack('grade')
#reading_score_by_grade_df.columns.droplevel(0)

reading_score                                 
grade                          10th       11th       12th        9th
school_name                                                         
Bailey High School        80.907183  80.945643  80.912451  81.303155
Cabrera High School       84.253219  83.788382  84.287958  83.676136
Figueroa High School      81.408912  80.640339  81.384863  81.198598
Ford High School          81.262712  80.403642  80.662338  80.632653
Griffin High School       83.706897  84.288089  84.013699  83.369193
Hernandez High School     80.660147  81.396140  80.857143  80.866860
Holden High School        83.324561  83.815534  84.698795  83.677165
Huang High School         81.512386  81.417476  80.305983  81.290284
Johnson High School       80.773431  80.616027  81.227564  81.260714
Pena High School          83.612000  84.335938  84.591160  83.807273
Rodriguez High School     80.629808  80.864811  80.376426  80.993127
Shelton High School       83.441964  84.373786  82.781671  84.122642
Thomas High School        84.254157  83.585542  83.831361  83.728850
Wilson High School        84.021452  83.764608  84.317673  83.939778
Wright High School        83.812757  84.156322  84.073171  83.833333

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [65]:
#school budget min & max values
print(school_summary_df ['Budget Per Student'].min())
print(school_summary_df ['Budget Per Student'].max())

# Create bins in which to place the different categories based on school spending
bins = [0, 585, 630, 645, 675]

#create labels for the bins
funding_brackets = ['Below 585', '585-630', '630-645', '545-675']

#create table with bins
school_summary_df['Budget Group'] = pd.cut(school_summary_df['Budget Per Student'], bins, labels = funding_brackets)
school_summary_df

budget_group = school_summary_df.groupby('Budget Group')
budget_group[['Average Math Score', 'Average Reading Score']].mean()


578.0
655.0


Average Math Score  Average Reading Score
Budget Group                                           
Below 585              83.455399              83.933814
585-630                81.899826              83.155286
630-645                78.518855              81.624473
545-675                76.997210              81.027843

## Scores by School Size

* Perform the same operations as above, based on school size.

In [68]:
#school size min & max values
print(school_summary_df ['Total Students'].min())
print(school_summary_df ['Total Students'].max())

# Create bins in which to place the different categories based on total no of students in each shool
bins = [0, 1000, 2000, 5000]

#create labels for the bins
school_size_brackets = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

#create table with bins
school_summary_df['School Size'] = pd.cut(school_summary_df['Total Students'], bins, labels = size_brackets)
school_summary_df

#Create groups with the school size in first column
school_size_group = school_summary_df.groupby('School Size Group')
school_size_group[['Average Math Score', 'Average Reading Score']].mean()

427
4976


Average Math Score  Average Reading Score
School Size Group                                            
Small (<1000)                83.821598              83.929843
Medium (1000-2000)           83.374684              83.864438
Large (2000-5000)            77.746417              81.344493

## Scores by School Type

* Perform the same operations as above, based on school type

In [72]:
## Create bins in which to place the different categories based on total no of students in each shool
#bins = ['[Charter]', '[District]']

##create labels for the bins
#district_brackets = ['[Charter]', '[District]']

##create table with bins
#school_summary_df['School District'] = pd.cut(school_summary_df['School Type'], bins, labels = district_brackets)
#school_summary_df

##Create groups with the school size in first column
#school_size_group = school_summary_df.groupby('School District')
#school_size_group[['Average Math Score', 'Average Reading Score']].mean()